In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [48]:
import pandas as pd

train = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
test = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")

In [49]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


In [50]:
train = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
test = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")


In [51]:
ndvi_columns = [col for col in train.columns if col.endswith('_N')]

X_train_full = train[ndvi_columns].interpolate(axis=1).bfill(axis=1).ffill(axis=1)
X_test_full = test[ndvi_columns].interpolate(axis=1).bfill(axis=1).ffill(axis=1)


In [52]:

def create_features(df):
    return pd.DataFrame({
        'mean': df.mean(axis=1),
        'std': df.std(axis=1),
        'min': df.min(axis=1),
        'max': df.max(axis=1),
        'range': df.max(axis=1) - df.min(axis=1),
        'median': df.median(axis=1),
        'q25': df.quantile(0.25, axis=1),
        'q75': df.quantile(0.75, axis=1)
    })

X_train_feat = create_features(X_train_full)
X_test_feat = create_features(X_test_full)


In [53]:
le = LabelEncoder()
y = le.fit_transform(train['class']) 


In [54]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_feat)
X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000, multi_class='multinomial')
model.fit(X_tr, y_tr)



LogisticRegression(max_iter=1000, multi_class='multinomial')

In [55]:
test_preds = model.predict(X_test_feat.to_numpy())

test_labels = le.inverse_transform(test_preds)

submission = pd.DataFrame({
    'ID': test['ID'],
    'class': test_labels
})

submission.to_csv("submission.csv", index=False)
